## 模糊作业2： 模糊控制表的求取
2112105068 韦青茂

### 作业描述
根据4.3.1例子中给出的热水瓶温度模糊控制的结构、I/O模糊化表、控制规则表如下，编程求出热水瓶温度模糊控制的控制表。

### python代码实现
#### 1. 首先定义输入\输出的模糊化表以及规则表

In [1]:
import numpy as np
NB,NS,Z,PS,PB = 0,1,2,3,4
# X
table_e = {
    # 'value':[-4, -3, -2, -1, 0, 1, 2, 3, 4],
    PB:[0,  0,  0,  0,  0,  0,  0, .5,  1],
    PS:[0,  0,  0,  0,  0, .5,  1, .5,  0],
    Z: [0,  0,  0, .5,  1, .5,  0,  0,  0],
    NS:[0, .5,  1, .5,  0,  0,  0,  0,  0],
    NB:[1, .5, .5,  0,  0,  0,  0,  0,  0],
}
# Y
table_DeltaE = {
    PB:[0,  0,  0,  0,  1],
    PS:[0,  0,  0,  1,  0],
    Z: [0,  0,  1,  0,  0],
    NS:[0,  1,  0,  0,  0],
    NB:[1,  0,  0,  0,  0],   
}
# Output
table_U = {
    PB:[0,  0,  0,  0,  0, .5,  1],
    PS:[0,  0,  0,  0,  1, .5,  0],
    Z: [0,  0, .5,  1, .5,  0,  0],
    NS:[0,  1, .5,  0,  0,  0,  0],
    NB:[1, .5,  0,  0,  0,  0,  0],
}

U_output = np.arange(-3,4)


table_rules = [
    [PB, PB, PS,  Z, NB],
    [PB, PB, PS, NS, NB],
    [PB, PS,  Z, NS, NB],
    [PB, PS, NS, NB, NB],
    [PB,  Z, NS, NB, NB],
]

# 将所有表转换成numpy矩阵
E = np.array([x[1] for x in sorted(table_e.items(),key=lambda kv:kv[0])])
dE = np.array([x[1] for x in sorted(table_DeltaE.items(),key=lambda kv:kv[0])])
U = np.array([x[1] for x in sorted(table_U.items(),key=lambda kv:kv[0])])
rules = np.array(table_rules)

#### 2. 以(x,y)=(-3,-2)为例的计算过程
2.1 分别取 $e$ 、$\Delta e$ 的I/O模糊化表中对应列的非0项

In [2]:
# x[1]=-3, y[0]=-2
X, Y = 1,0 
# 表E中不为0的行：
idx_E = E[:,X] != 0
# 表dE中不为0的行：
idx_dE = dE[:,Y] != 0
print(idx_E)
print(idx_dE)

[ True  True False False False]
[ True False False False False]


2.2 由规则表，得到有效规则的结果

In [3]:
eff_rules = rules[idx_dE, idx_E]
print(eff_rules)

[4 4]


2.3 对所有有效规则计算相应的输出模糊值C

In [4]:
mu_E = E[:,X][idx_E]
mu_dE = dE[:,Y][idx_dE]
mu_U = U[eff_rules]
# 利用矩阵间最小值运算得到最小值
min_temp = np.minimum(mu_dE,mu_E)
_C = np.minimum((min_temp).reshape(len(min_temp),1), mu_U)
print(_C)

[[0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.5 0.5]]


2.4 对所有C'取最大值得到控制输出的模糊量

In [5]:
C = np.max(_C, axis=0)
print(C)

[0.  0.  0.  0.  0.  0.5 0.5]


2.5 利用重心法进行反模糊化

In [6]:
z = np.vdot(U_output, C)/np.sum(C)
print(z)

2.5


2.6 根据量化公式进行约分

In [7]:
u = np.trunc(z+np.sign(Z)*0.4)
print(u)

2.0


#### 3. 对控制表的所有位置进行上述过程的计算

In [9]:
out = np.zeros([9,5])
for X in range(9):
    for Y in range(5):
        # x不为0的项：
        idx_E = E[:,X] != 0
        # y不为0的项：
        idx_dE = dE[:,Y] != 0
        # 有效规则的then
        eff_rules = rules[idx_dE, idx_E]
        # 应用规则后, 利用矩阵运算求矩阵间最小值
        mu_E = E[:,X][E[:,X]>0]
        mu_dE = dE[:,Y][dE[:,Y]>0]
        mu_U = U[eff_rules]
        min_temp = np.minimum(mu_dE,mu_E)
        # 对每条有效规则求Cn
        _C = np.minimum((min_temp).reshape(len(min_temp),1), mu_U)
        # 取Cn的最大值
        C = np.max(_C, axis=0)
        # 重心法进行反模糊化
        out[X,Y] = np.vdot(U_output, C) / np.sum(C)
# 根据量化公式约分
print(np.trunc(out+np.sign(out)*0.5))

[[ 3.  3.  3.  3.  3.]
 [ 3.  3.  2.  2.  1.]
 [ 3.  3.  2.  2.  1.]
 [ 2.  2.  1.  0. -1.]
 [ 1.  1.  0. -2. -2.]
 [ 1.  0. -1. -2. -2.]
 [ 0. -2. -2. -3. -3.]
 [-1. -2. -2. -3. -3.]
 [-3. -3. -3. -3. -3.]]


![](./%E8%A1%A8.svg)